In [1]:
!nvidia-smi

Tue Jun 11 10:19:26 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    29W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Run this cell first to prepare your google drive

In [2]:
import os

# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

# Create a folder to save the output
#if not os.path.isdir("/content/drive/My Drive/CodeTransformer"):
#  os.mkdir("/content/drive/My Drive/CodeTransformer")
#else:
#  print("Directory already exists, skipping...")
  
# Change working directory to this created folder
#os.chdir("/content/drive/My Drive/CodeTransformer")

# If the git does not exist, clone it, else open the folder
if not os.path.isdir("./Deep-Learning"):
  print("Initial clone of git repo...")
  !git clone https://github.com/jverbraeken/Deep-Learning.git
  os.chdir("./Deep-Learning")
else:
  print("Folder exists, pulling git repo...")
  os.chdir("./Deep-Learning")
  !git pull

# Create folders for Tensor2Tensor to operate in
if not os.path.isdir("./data"):
  print("Creating data folder...")
  os.mkdir("./data")

if not os.path.isdir("./tmp"):
  print("Creating tmp folder...")
  os.mkdir("./tmp")

if not os.path.isdir("./output"):
  print("Creating output folder...")
  os.mkdir("./output")
  
# Should print a list of folders and files from the git
!ls
  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Initial clone of git repo...
Cloning into 'Deep-Learning'...
remote: Enumerating objects: 234, done.
remote: Total 234 (delta 0), reused 0 (delta 0), pack-reused 234
Receiving objects: 100% (234/234), 344.90 MiB | 46.74 MiB/s, done.
Resolving deltas: 100% (102/102), done.
Creating data folder...
Creating tmp folder...
Creating output folder...
data		    output			 transformer
data.7z		    prepare_transformer_data.py  transformer_data
Evaluation	    preprocess		

# (Optional) Load model from drive

In [3]:
!unzip /content/drive/My\ Drive/model.zip

Archive:  /content/drive/My Drive/model.zip
  inflating: output/events.out.tfevents.1556782933.66fae7786de6  
  inflating: output/model.ckpt-743000.meta  
  inflating: output/events.out.tfevents.1556735957.82ba9f0dd0f8  
  inflating: output/model.ckpt-743000.data-00000-of-00002  
   creating: output/eval/
  inflating: output/eval/events.out.tfevents.1556700650.b5744b070ce1  
  inflating: output/eval/events.out.tfevents.1556720857.82ba9f0dd0f8  
  inflating: output/eval/events.out.tfevents.1556703588.82ba9f0dd0f8  
  inflating: output/eval/events.out.tfevents.1556799355.66fae7786de6  
  inflating: output/eval/events.out.tfevents.1556703304.82ba9f0dd0f8  
  inflating: output/eval/events.out.tfevents.1556736137.82ba9f0dd0f8  
  inflating: output/eval/events.out.tfevents.1556662560.e2c3fc9cbd7f  
  inflating: output/eval/events.out.tfevents.1556662861.e2c3fc9cbd7f  
  inflating: output/eval/events.out.tfevents.1556783237.66fae7786de6  
  inflating: output/eval/events.out.tfevents.155665449

# Run tensorboard

In [0]:
LOG_DIR = './output'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
# Install
! npm install -g localtunnel

# Tunnel port 6006 (TensorBoard assumed running)
get_ipython().system_raw('lt --port 6006 >> url.txt 2>&1 &')

In [0]:
!cat url.txt

# Run this cell to generate the data for the Transformer

In [4]:
!t2t-datagen --t2t_usr_dir=./transformer --data_dir=./data --tmp_dir=./tmp --problem=code_transformer


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Importing user module transformer from path /content/Deep-Learning
INFO:tensorflow:Generating problems:
    code:
      * code_transformer
INFO:tensorflow:Generating data for code_transformer.
:::MLPv0.5.0 transformer 1560248721.629471064 (/usr/local/lib/python2.7/dist-packages/tensor2tensor/data_generators/text_problems.py:306) preproc_tokenize_training
INFO:tensorflow:Downloading https://github.com/jverbraeken/Deep-Learning/blob/master/transformer_data/train.tgz?raw=true to ./tmp/train.tgz?raw=true
100% completed
INFO:tensorflow:Successfully downloaded train.tgz?raw=true, 76744618 bytes.
INFO:tensorflow:Generating vocab file: ./data/vocab.code_transformer.8192.subwords
INFO:tensorflow:Generating vocab from: [['https://github.com/jverbra

# Run this cell to train the Transformer

In [0]:
!t2t-trainer --generate_data --t2t_usr_dir=./transformer --data_dir=./data --output_dir=./output --problem=code_transformer --model=transformer --hparams_set=transformer_base_single_gpu --hparams='batch_size=1024' --keep_checkpoint_max=1 --train_steps=750000

# Upload trained model to drive

In [0]:
#import shutil
#!zip -r ./model.zip ./output
shutil.move("./model.zip", "/content/drive/My Drive")
drive.mount('/content/drive', force_remount=True)

In [0]:
!ls -lh

# Run decoder

In [5]:
!t2t-decoder --t2t_usr_dir=./transformer --data_dir=./data --output_dir=./output --problem=code_transformer --model=transformer --hparams_set=transformer_base_single_gpu --hparams='batch_size=1024' --decode_hparams="beam_size=4,alpha=0.6" --decode_from_file="./sampled.cd" --decode_to_file="./sampled_model.en"


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Importing user module transformer from path /content/Deep-Learning
INFO:tensorflow:Overriding hparams in transformer_base_single_gpu with batch_size=1024
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 20, '_task_type': None, '_train_distribute': None, '_is_chief': True, '_cluster_spe